# Imports

In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.insert(0, "lib/")
from data.coco_dataset import CocoDataset
from utils.preprocess_sample import preprocess_sample
from utils.collate_custom import collate_custom
from utils.utils import to_cuda_variable
import utils.result_utils as result_utils
from utils.json_dataset_evaluator import evaluate_boxes,evaluate_masks
from model.detector import detector
from utils.blob import get_rois_blob
from utils.multilevel_rois import add_multilevel_rois_for_test

torch_ver = torch.__version__[:3]

# Parameters

In [2]:
# Pretrained model
# # https://s3-us-west-2.amazonaws.com/detectron/35859007/12_2017_baselines/e2e_mask_rcnn_R-50-FPN_2x.yaml.01_49_07.By8nQcCH/output/train/coco_2014_train%3Acoco_2014_valminusminival/generalized_rcnn/model_final.pkl
# arch='resnet50'
# pretrained_model_file = 'files/trained_models/mask/e2e_mask_rcnn_R-50-FPN_2x.pkl'

# https://s3-us-west-2.amazonaws.com/detectron/35861858/12_2017_baselines/e2e_mask_rcnn_R-101-FPN_2x.yaml.02_32_51.SgT4y1cO/output/train/coco_2014_train:coco_2014_valminusminival/generalized_rcnn/model_final.pkl
arch='resnet101'
pretrained_model_file = 'files/trained_models/mask/e2e_mask_rcnn_R-101-FPN_2x.pkl'

# COCO minival2014 dataset path
coco_ann_file='datasets/data/coco/annotations/instances_minival2014.json'
img_dir='datasets/data/coco/val2014'

# Create dataset

In [3]:
dataset = CocoDataset(ann_file=coco_ann_file,img_dir=img_dir,
                       sample_transform=preprocess_sample(target_sizes=[800],fpn_on=True))
dataloader = DataLoader(dataset, batch_size=1, # only batch_size=1 is supported by now
                        shuffle=False, num_workers=0, collate_fn=collate_custom)

loading annotations into memory...
Done (t=0.62s)
creating index...
index created!


# Create detector model

In [4]:
model = detector(arch=arch,
                 detector_pkl_file=pretrained_model_file,
                 conv_body_layers=['conv1','bn1','relu','maxpool','layer1','layer2','layer3','layer4'],
                 conv_head_layers='two_layer_mlp',
                 fpn_layers=['layer1','layer2','layer3','layer4'],
                 fpn_extra_lvl=True,
                 roi_height=7,
                 roi_width=7,
                 roi_spatial_scale=[0.25,0.125,0.0625,0.03125],
                 roi_sampling_ratio=2,
                 use_rpn_head = True,
                 use_mask_head = True,
                 mask_head_type = '1up4convs')
model = model.cuda()

Loading pretrained weights:
-> loading conv. body weights
-> loading output head weights
-> loading rpn head weights
-> loading mask head weights
-> loading 1up4convs mask head weights
-> loading FPN lateral weights
-> loading two layer mlp conv head...


# Evaluate

In [5]:
# Create data structure to store results
all_boxes, all_segms, all_keyps = result_utils.empty_results(dataset.num_classes, len(dataset)) 
# (only all_boxes will be used for fast RCNN)

In [6]:
# Compute detections for whole dataset
for i, batch in enumerate(dataloader):
    batch = to_cuda_variable(batch)
    # forward pass
    if torch_ver=="0.4": # handle change in "volatile"
        with torch.no_grad():
            class_scores,bbox_deltas,rois,img_features=model(batch['image'],
                                                             scaling_factor=batch['scaling_factors'])   
    else:
        class_scores,bbox_deltas,rois,img_features=model(batch['image'],
                                                             scaling_factor=batch['scaling_factors'])   
    # postprocess output:
    # - convert coordinates back to original image size, 
    # - treshold proposals based on score,
    # - do NMS.
    scores_final, boxes_final, boxes_per_class = result_utils.postprocess_output(rois,
                                                                    batch['scaling_factors'],
                                                                    batch['original_im_size'],
                                                                    class_scores,
                                                                    bbox_deltas)
    if len(boxes_final)==0:
        continue
        
    # compute masks
    boxes_final_multiscale = add_multilevel_rois_for_test({'rois': boxes_final*batch['scaling_factors']},'rois')
    boxes_final_multiscale_th = []
    for k in boxes_final_multiscale.keys():
        if len(boxes_final_multiscale[k])>0 and 'rois_fpn' in k:
            boxes_final_multiscale_th.append(Variable(torch.cuda.FloatTensor(boxes_final_multiscale[k])))
        elif len(boxes_final_multiscale[k])==0 and 'rois_fpn' in k:
            boxes_final_multiscale_th.append(None)
    rois_idx_restore_th = Variable(torch.cuda.FloatTensor(boxes_final_multiscale['rois_idx_restore_int32']))
    masks=model.mask_head(img_features,boxes_final_multiscale_th,rois_idx_restore_th.long())
    # postprocess mask output:
    h_orig = int(batch['original_im_size'].squeeze()[0].data.cpu().numpy().item())
    w_orig = int(batch['original_im_size'].squeeze()[1].data.cpu().numpy().item())
    cls_segms = result_utils.segm_results(boxes_per_class, masks.cpu().data.numpy(), boxes_final, h_orig, w_orig,
                                          M=28) # M: Mask R-CNN resolution
    
    # store results
    result_utils.extend_results(i, all_boxes, boxes_per_class)
    result_utils.extend_results(i, all_segms, cls_segms)
    
    if i%100==0:
        print("{}/{}".format(i+1,len(dataset)))

print('Done!')

1/5000
101/5000
201/5000
301/5000
401/5000
501/5000
601/5000
701/5000
801/5000
901/5000
1001/5000
1101/5000
1201/5000
1301/5000
1401/5000
1501/5000
1601/5000
1701/5000
1801/5000
1901/5000
2001/5000
2101/5000
2201/5000
2301/5000
2401/5000
2501/5000
2601/5000
2701/5000
2801/5000
2901/5000
3001/5000
3101/5000
3201/5000
3301/5000
3401/5000
3501/5000
3601/5000
3701/5000
3801/5000
3901/5000
4001/5000
4101/5000
4201/5000
4301/5000
4401/5000
4501/5000
4601/5000
4701/5000
4801/5000
4901/5000
Done!


In [7]:
# # Save detection and segmentation results
np.save('files/results/all_boxes_segms_mask.npy',{'all_boxes': all_boxes, 'all_segms': all_segms})

In [8]:
# Compute evaluation metrics
coco_box_eval = evaluate_boxes(json_dataset=dataset.coco, 
                           all_boxes=all_boxes, 
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=1.47s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=36.72s).
Accumulating evaluation results...
DONE (t=5.45s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.619
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.448
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.235
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.442
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.539
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.329
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.510
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.533
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [9]:
coco_segm_eval = evaluate_masks(json_dataset=dataset.coco, 
                           all_boxes=all_boxes,
                           all_segms=all_segms,
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=2.87s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=40.29s).
Accumulating evaluation results...
DONE (t=5.37s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.364
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.585
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.387
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.166
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.392
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.540
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.303
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.459
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.478
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1